In [1]:
import os
import shutil
import sys
sys.path.append('Network/')
sys.path.append('NGEV/')
sys.path.append('Matrix/')
sys.path.append('optimizationProgram/')

import makeTSnet as mts
import readNetwork as rn
import makeConstMat as mcm
import readSparseMat as rsm
import makeODdemand as mod
import NGEV as ngev

In [2]:
# 実験するネットワークの名前を入れましょう
folder_name = '_sampleData'
network_name = 'Sample'

In [3]:
# 必要なファイルがそろっているかを確認するセル(エラーが出たら必要なファイルを確認して用意すること)
root = os.path.dirname(os.path.abspath('.'))
root = os.path.join(root, folder_name, network_name)
print(root)
files = os.listdir(root)

files_is_ready = 0

if 'netname_net.tntp'.replace('netname', network_name) not in files:
    print('netname_net.tntp file is not exist.')
    files_is_ready = -1
if 'netname_node.tntp'.replace('netname', network_name) not in files:
    print('netname_node.tntp file is no exist.')
    files_is_ready = -1
if 'netname_trips.tntp'.replace('netname', network_name) not in files:
    print('netname_trips.tntp file is not exist.')
    files_is_ready = -1



scenario_set = [f for f in files if os.path.isdir(os.path.join(root, f))]
for scene in scenario_set:
    scene_files = os.listdir(os.path.join(root, scene))
    if 'netname_vu.tntp'.replace('netname', network_name) not in scene_files:
        print(scene + '/netname_vu.tntp file is not exist.')
        files_is_ready = -1


if files_is_ready == -1:
    print('FILE NOT READY ERROR!!!')

print(scenario_set)

c:\Users\tsuba\Documents\TNPandMS_experCode\_sampleData\Sample
['Scenario_0']


### ここから実験コード

In [4]:
# 必要データの生成
# capacity_scale: 道路容量を何倍にするか．時空間ネットワーク作成時に利用
# non_trip_cost: tripを実行しない場合のコスト．仮想ネットワーク作成時に利用
capacity_scale = 0.75
non_trip_cost = 20.0

In [5]:
for scenario in scenario_set:

    print('scenario: ', scenario, '\n')

    scenario_root = os.path.join(root, scenario)

    # 時空間ネットワークを作成
    original_links = rn.read_net(os.path.join(root, 'netname_net.tntp'.replace('netname', network_name)))
    num_zones = rn.read_num_zones(os.path.join(root, 'netname_net.tntp'.replace('netname', network_name)))
    ftn = rn.read_ftn(os.path.join(root, 'netname_net.tntp'.replace('netname', network_name)))
    original_nodes = rn.read_node(os.path.join(root, 'netname_node.tntp'.replace('netname', network_name)))
    num_times = rn.read_num_times(os.path.join(scenario_root, 'netname_vu.tntp'.replace('netname', network_name)))
    [TS_links, TS_nodes] = mts.make_TS_net(original_links, original_nodes, num_times, capacity_scale)
    num_ts_links = len(TS_links)
    num_in_links = len(TS_nodes[TS_nodes['time'] != num_times-1])

    # 時空間ネットワークを保存
    temp_root = os.path.join(scenario_root, 'TS_net')
    os.makedirs(temp_root, exist_ok=True)
    rn.write_net(os.path.join(temp_root, 'netname_ts_net.tntp'.replace('netname', network_name)), TS_links, num_zones*2, len(TS_nodes), ftn, len(TS_links))
    rn.write_node(os.path.join(temp_root, 'netname_ts_node.tntp'.replace('netname', network_name)), TS_nodes)

    # 時空間ネットワークに関するメモリを解放
    del TS_links, TS_nodes






    # 仮想ネットワークを作成
    [vehicle_info, user_info] = rn.read_vu(os.path.join(scenario_root, 'netname_vu.tntp'.replace('netname', network_name)))
    [vehicle_scost, user_scost] = rn.read_scost(os.path.join(scenario_root, 'netname_scost.tntp'.replace('netname', network_name)))
    [vehicle_total_flow, user_total_flow] = rn.read_tflow(os.path.join(scenario_root, 'netname_tflow.tntp'.replace('netname', network_name)))

    num_price_index = 0
    for vehicle_num in vehicle_info.keys():
        for state in vehicle_info[vehicle_num].index:
            if num_price_index < max(vehicle_info[vehicle_num]['price_index'][state]):
                num_price_index = max(vehicle_info[vehicle_num]['price_index'][state])

    virtual_root = os.path.join(scenario_root, 'virtual_net')
    const_root = os.path.join(scenario_root, 'constMat')

    # 車両側の仮想ネットワーク
    for vehicle_num in vehicle_info.keys():

        # 仮想ネットワークを作成
        [VTS_links, VTS_nodes] = mts.make_vehicle_net(original_links, original_nodes, num_zones, num_times, vehicle_info[vehicle_num], vehicle_scost[vehicle_num], non_trip_cost)
        os.makedirs(os.path.join(virtual_root, 'vehicle', str(vehicle_num)), exist_ok=True)
        rn.write_net(os.path.join(virtual_root, 'vehicle', str(vehicle_num), 'netname_vir_net.tntp'.replace('netname', network_name)), VTS_links, num_zones*2, len(VTS_nodes), ftn, len(VTS_links))
        rn.write_node(os.path.join(virtual_root, 'vehicle', str(vehicle_num), 'netname_vir_node.tntp'.replace('netname', network_name)), VTS_nodes)

        # 係数行列を作成
        TNP_constMat = mcm.make_TNP_constMat(VTS_links, num_ts_links)
        MSV_constMat = mcm.make_MSV_constMat(VTS_links, VTS_nodes, vehicle_info[vehicle_num], num_price_index, num_ts_links, num_in_links)
        incMat = mcm.make_incMat(VTS_links, VTS_nodes)
        os.makedirs(os.path.join(const_root, 'vehicle', str(vehicle_num)), exist_ok=True)
        rsm.write_sparse_mat(os.path.join(const_root, 'vehicle', str(vehicle_num), 'TNP_constMat'), TNP_constMat)
        rsm.write_sparse_mat(os.path.join(const_root, 'vehicle', str(vehicle_num), 'MSV_constMat'), MSV_constMat)
        rsm.write_sparse_mat(os.path.join(const_root, 'vehicle', str(vehicle_num), 'incidenceMat'), incMat)

        # OD需要を作成
        vehicle_trips = mod.make_trips_random(num_zones, vehicle_total_flow[vehicle_num])
        rn.write_trips(os.path.join(virtual_root, 'vehicle', str(vehicle_num), 'netname_vir_trips.tntp'.replace('netname', network_name)), vehicle_trips, num_zones*2, vehicle_total_flow[vehicle_num])



    del VTS_links, VTS_nodes


    # 利用者側の仮想ネットワーク
    for user_num in user_info.keys():

        # 仮想ネットワークを作成
        [UTS_links, UTS_nodes] = mts.make_user_net(original_links, original_nodes, num_zones, num_times, user_info[user_num], user_scost[user_num], non_trip_cost)
        os.makedirs(os.path.join(virtual_root, 'user', str(user_num)), exist_ok=True)
        rn.write_net(os.path.join(virtual_root, 'user', str(user_num), 'Sample_vir_net.tntp'), UTS_links, num_zones*2, len(UTS_nodes), 1, len(UTS_links))
        rn.write_node(os.path.join(virtual_root, 'user', str(user_num), 'Sample_vir_node.tntp'), UTS_nodes)

        # 係数行列を作成
        MSU_constMat = mcm.make_MSU_constMat(UTS_links, UTS_nodes, num_price_index, num_ts_links, num_in_links)
        incMat = mcm.make_incMat(UTS_links, UTS_nodes)
        os.makedirs(os.path.join(const_root, 'user', str(user_num)), exist_ok=True)
        rsm.write_sparse_mat(os.path.join(const_root, 'user', str(user_num), 'MSU_constMat'), MSU_constMat)
        rsm.write_sparse_mat(os.path.join(const_root, 'user', str(user_num), 'incidenceMat'), incMat)

        # OD需要を作成
        user_trips = mod.make_trips_random(num_zones, user_total_flow[user_num])
        rn.write_trips(os.path.join(virtual_root, 'user', str(user_num), 'netname_vir_trips.tntp'.replace('netname', network_name)), user_trips, num_zones*2, user_total_flow[user_num])


    del UTS_links, UTS_nodes


    print('finish making virtual_network !!!!!!\n\n\n')







    # 時空間ネットワークを読み込む
    TS_links = rn.read_net(os.path.join(scenario_root, 'TS_net', 'netname_ts_net.tntp'.replace('netname', network_name)))
    # print(TS_links)
    
    veh_root = os.path.join(scenario_root, 'virtual_net', 'vehicle')
    veh_files = os.listdir(veh_root)
    user_root = os.path.join(scenario_root, 'virtual_net', 'user')
    user_files = os.listdir(user_root)


    # -----------------車両側の仮想ネットワーク情報を追加-------------------------------------------------------------------
    veh_links = {}
    veh_nodes = {}
    veh_trips = {}
    for file in veh_files:
        veh_links[int(file)] = rn.read_net(veh_root + '\\' + file + '\\netname_vir_net.tntp'.replace('netname', network_name))
        veh_nodes[int(file)] = rn.read_node(veh_root + '\\' + file + '\\netname_vir_node.tntp'.replace('netname', network_name))
        veh_trips[int(file)] = rn.read_trips(veh_root + '\\' + file + '\\netname_vir_trips.tntp'.replace('netname', network_name))

        # nodes の要らない情報を削除
        keys = veh_nodes[int(file)].columns
        veh_nodes[int(file)].drop(keys, axis=1, inplace=True)
        veh_nodes[int(file)]['theta'] = 1.0

        # links の要らない情報を削除
        keys = veh_links[int(file)].columns
        for key in keys:
            if key == 'init_node' or key == 'term_node' or key == 'free_flow_time':
                continue
            else:
                veh_links[int(file)].drop(key, axis=1, inplace=True)
        veh_links[int(file)]['alpha'] = 1.0

    # print(veh_links)
    # print(veh_nodes)
    # print(veh_trips)

    # -----------------利用者側の仮想ネットワーク情報を追加-------------------------------------------------------------------
    user_links = {}
    user_nodes = {}
    user_trips = {}
    for file in user_files:
        user_links[int(file)] = rn.read_net(user_root + '\\' + file + '\\netname_vir_net.tntp'.replace('netname', network_name))
        user_nodes[int(file)] = rn.read_node(user_root + '\\' + file + '\\netname_vir_node.tntp'.replace('netname', network_name))
        user_trips[int(file)] = rn.read_trips(user_root + '\\' + file + '\\netname_vir_trips.tntp'.replace('netname', network_name))

        # nodes の要らない情報を削除
        keys = user_nodes[int(file)].columns
        user_nodes[int(file)].drop(keys, axis=1, inplace=True)
        user_nodes[int(file)]['theta'] = 1.0

        # links の要らない情報を削除
        keys = user_links[int(file)].columns
        for key in keys:
            if key == 'init_node' or key == 'term_node' or key == 'free_flow_time':
                continue
            else:
                user_links[int(file)].drop(key, axis=1, inplace=True)
        user_links[int(file)]['alpha'] = 1.0


    # -----------------制約条件の係数行列を取得-------------------------------------------------------------------

    veh_root = os.path.join(scenario_root, 'constMat', 'vehicle')
    veh_files = os.listdir(veh_root)
    user_root = os.path.join(scenario_root, 'constMat', 'user')
    user_files = os.listdir(user_root)

    # 車両側の行列を取得
    TNP_constMat = {}
    MSV_constMat = {}
    for file in veh_files:
        TNP_constMat[int(file)] = rsm.read_sparse_mat(veh_root + '\\' + file + '\TNP_constMat')
        MSV_constMat[int(file)] = rsm.read_sparse_mat(veh_root + '\\' + file + '\MSV_constMat')

    # 利用者側の行列を取得
    MSU_constMat = {}
    for file in user_files:
        MSU_constMat[int(file)] = rsm.read_sparse_mat(user_root + '\\' + file + '\MSU_constMat')

    output_root = os.path.join(scenario_root, 'result')
    os.makedirs(output_root, exist_ok=True)

    ngev.NGEV_TNPandMS(veh_nodes, veh_links, veh_trips, TNP_constMat, MSV_constMat, user_nodes, user_links, user_trips, MSU_constMat, TS_links, output_root+'\\result.csv')

    shutil.rmtree(os.path.join(scenario_root, 'constMat'))
    shutil.rmtree(os.path.join(scenario_root, 'TS_net'))
    shutil.rmtree(os.path.join(scenario_root, 'virtual_net'))


scenario:  Scenario_0 

finish making virtual_network !!!!!!






start Accel Gradient Projection Method (FISTA with backtracking)!
iteration: 0  now_sol: [0. 0. 0. 0. 0.]  convergence: 1.099453459881791
iteration: 1  now_sol: [0. 0. 0. 0. 0.]  convergence: 1.0916635804650863
iteration: 2  now_sol: [0. 0. 0. 0. 0.]  convergence: 0.6875397375771003
iteration: 3  now_sol: [0. 0. 0. 0. 0.]  convergence: 0.3571437078642674
iteration: 4  now_sol: [0. 0. 0. 0. 0.]  convergence: 0.7665918242684568
iteration: 5  now_sol: [0. 0. 0. 0. 0.]  convergence: 0.009657520880478367
finish accel gradient projection method
sol:  [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         1.81051401 5.76377476 0.99508468 0.75859759
 1.88967166 0.         0.         0.         0.         0.
 1.53814799 1.80374439 1.63469414 0.33861345 0.         0.
 0.         0.      